by uramoon@kw.ac.kr <br>
원본 출처: https://github.com/rickiepark/deep-learning-with-python-notebooks <a href="https://github.com/rickiepark/deep-learning-with-python-notebooks/blob/master/LICENSE">(MIT License)</a>

In [1]:
import keras
keras.__version__

'3.5.0'

# 신경망과의 두 번째 만남

이 노트북은 [케라스 창시자에게 배우는 딥러닝](https://tensorflow.blog/케라스-창시자에게-배우는-딥러닝/) 책의 2장 1절의 코드 예제입니다. 책에는 더 많은 내용과 그림이 있습니다. 이 노트북에는 소스 코드에 관련된 설명만 포함합니다. 이 노트북의 설명은 케라스 버전 2.2.2에 맞추어져 있습니다. 케라스 최신 버전이 릴리스되면 노트북을 다시 테스트하기 때문에 설명과 코드의 결과가 조금 다를 수 있습니다.

----

케라스 파이썬 라이브러리를 사용하여 손글씨 숫자 분류를 학습하는 구체적인 신경망 예제를 살펴보겠습니다. 케라스나 비슷한 라이브러리를 사용한 경험이 없다면 당장은 이 첫 번째 예제를 모두 이해하지 못할 것입니다. 아직 케라스를 설치하지 않았을지도 모릅니다. 괜찮습니다. 다음 장에서 이 예제를 하나하나 자세히 설명합니다. 코드가 좀 이상하거나 요술처럼 보이더라도 너무 걱정하지 마세요. 일단 시작해 보겠습니다.

여기에서 풀려고 하는 문제는 흑백 손글씨 숫자 이미지(28x28 픽셀)를 10개의 범주(0에서 9까지)로 분류하는 것입니다. 머신 러닝 커뮤니티에서 고전으로 취급받는 데이터셋인 MNIST를 사용하겠습니다. 이 데이터셋은 머신 러닝의 역사만큼 오래되었고 많은 연구에 사용되었습니다. 이 데이터셋은 1980년대에 미국 국립표준기술연구소에서 수집한 6만 개의 훈련 이미지와 1만 개의 테스트 이미지로 구성되어 있습니다. MNIST 문제를 알고리즘이 제대로 작동하는지 확인하기 위한 딥러닝계의 ‘Hello World’라고 생각해도 됩니다. 머신 러닝 기술자가 되기까지 연구 논문이나 블로그 포스트 등에서 MNIST를 보고 또 보게 될 것입니다.

MNIST 데이터셋은 넘파이 배열 형태로 케라스에 이미 포함되어 있습니다:

In [2]:
# TODO: Keras에서 제공하는 MNIST 데이터셋 불러오기
from keras.datasets import mnist

(train_images, train_labels), (test_images, test_labels) = mnist.load_data()

11490434/11490434 ━━━━━━━━━━━━━━━━━━━━ 0s 0us/step


`train_images`와 `train_labels`가 모델이 학습해야 할 훈련 세트를 구성합니다. 모델은 `test_images`와 `test_labels`로 구성된 테스트 세트에서 테스트될 것입니다. 이미지는 넘파이 배열로 인코딩되어 있고 레이블은 0에서부터 9까지의 숫자 배열입니다. 이미지와 레이블은 일대일 관계를 가집니다.



작업 순서는 다음과 같습니다. 먼저 훈련 데이터 `train_images`와 `train_labels`를 네트워크에 주입합니다. 그러면 네트워크는 이미지와 레이블을 연관시킬 수 있도록 학습됩니다. 마지막으로 `test_images`에 대한 예측을 네트워크에게 요청합니다. 그리고 이 예측이 `test_labels`와 맞는지 확인할 것입니다.

신경망을 만들어 보겠습니다.

In [3]:
from keras import models
from keras import layers

network = models.Sequential()
network.add(layers.Input(shape=(28 * 28,)))
network.add(layers.Dense(512, activation='relu'))
network.add(layers.Dense(10, activation='softmax'))

In [4]:
# 네트워크의 구조 보기
network.summary()

Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ dense (Dense)                        │ (None, 512)                 │         401,920 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_1 (Dense)                      │ (None, 10)                  │           5,130 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 407,050 (1.55 MB)

 Trainable params: 407,050 (1.55 MB)

 Non-trainable params: 0 (0.00 B)

신경망의 핵심 구성 요소는 일종의 데이터 처리 필터라고 생각할 수 있는 층입니다. 어떤 데이터가 들어가면 더 유용한 형태로 출력됩니다. 조금 더 구체적으로 층은 주어진 문제에 더 의미 있는 표현을 입력된 데이터로부터 추출합니다. 대부분의 딥러닝은 간단한 층을 연결하여 구성되어 있고, 점진적으로 데이터를 정제하는 형태를 띠고 있습니다. 딥러닝 모델은 데이터 정제 필터(층)가 연속되어 있는 데이터 프로세싱을 위한 여과기와 같습니다.

이 예에서는 조밀하게 연결된 (또는 완전 연결된) 신경망 층인 `Dense` 층 2개가 연속되어 있습니다. 두 번째 (즉, 마지막) 층은 10개의 확률 점수가 들어 있는 배열(모두 더하면 1입니다)을 반환하는 소프트맥스 층입니다. 각 점수는 현재 숫자 이미지가 10개의 숫자 클래스 중 하나에 속할 확률입니다.

신경망이 훈련 준비를 마치기 위해서 컴파일 단계에 포함될 세 가지가 더 필요합니다:

* 손실 함수 : 훈련 데이터에서 신경망의 성능을 측정하는 방법으로 네트워크가 옳은 방향으로 학습될 수 있도록 도와 줍니다.
* 옵티마이저: 입력된 데이터와 손실 함수를 기반으로 네트워크를 업데이트하는 메커니즘입니다.
* 훈련과 테스트 과정을 모니터링할 지표 : 여기에서는 정확도(정확히 분류된 이미지의 비율)만 고려하겠습니다.

손실 함수와 옵티마이저의 정확한 목적은 이어지는 두 개의 장에서 자세히 설명하겠습니다.

In [5]:
network.compile(optimizer='rmsprop',
                loss='categorical_crossentropy',
                metrics=['accuracy'])

훈련을 시작하기 전에 데이터를 네트워크에 맞는 크기로 바꾸고 모든 값을 0과 1 사이로 스케일을 조정합니다. 예를 들어, 앞서 우리의 훈련 이미지는 `[0, 255]` 사이의 값인 `uint8` 타입의 `(60000, 28, 28)` 크기를 가진 배열로 저장되어 있습니다. 이 데이터를 0과 1 사이의 값을 가지는 `float32` 타입의 `(60000, 28 * 28)` 크기의 배열로 바꿉니다.

In [10]:
# TODO: [0, 255]의 이차원 정수 배열로 표현된 이미지를
# [0, 1]의 일차원 실수 배열로 변환하세요. (Normalization, Scaling or 정규화)

norm_train_images = train_images.reshape((60000, 28 * 28))
norm_train_images = norm_train_images.astype('float32')/255.0
norm_test_images = test_images.reshape((10000,28*28))
norm_test_images = norm_test_images.astype('float32')/255.0

또한, 레이블을 범주형으로 인코딩해야 합니다. 이 단계는 3장에서 자세히 설명하겠습니다:

In [11]:
# TODO: 레이블을 범주형으로 변환

from tensorflow.keras.utils import to_categorical

cat_train_labels = to_categorical(train_labels)
cat_test_labels = to_categorical(test_labels)

이제 신경망을 훈련시킬 준비가 되었습니다. 케라스에서는 `fit` 메서드를 호출하여 훈련 데이터에 모델을 학습시킵니다:

In [12]:
network.fit(norm_train_images, cat_train_labels, epochs=5, batch_size=128)

Epoch 1/5
469/469 ━━━━━━━━━━━━━━━━━━━━ 5s 9ms/step - accuracy: 0.8705 - loss: 0.4457
Epoch 2/5
469/469 ━━━━━━━━━━━━━━━━━━━━ 5s 9ms/step - accuracy: 0.9663 - loss: 0.1147
Epoch 3/5
469/469 ━━━━━━━━━━━━━━━━━━━━ 6s 10ms/step - accuracy: 0.9791 - loss: 0.0712
Epoch 4/5
469/469 ━━━━━━━━━━━━━━━━━━━━ 4s 8ms/step - accuracy: 0.9854 - loss: 0.0511
Epoch 5/5
469/469 ━━━━━━━━━━━━━━━━━━━━ 5s 11ms/step - accuracy: 0.9898 - loss: 0.0353


훈련하는 동안 두 개의 정보가 출력됩니다. 훈련 데이터에 대한 네트워크의 손실과 정확도입니다.

훈련 데이터에 대해 0.989(98.9%)의 정확도를 금방 달성합니다. 이제 테스트 세트에서도 모델이 잘 작동하는지 확인해 보겠습니다:

In [13]:
test_loss, test_acc = network.evaluate(norm_test_images, cat_test_labels)

313/313 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - accuracy: 0.9763 - loss: 0.0805


In [14]:
print('test_acc:', test_acc)

test_acc: 0.9804999828338623


테스트 세트의 정확도는 97.8%로 나왔습니다. 훈련 세트 정확도보다는 약간 낮습니다. 훈련 정확도와 테스트 정확도 사이의 차이는 과대적합 때문입니다. 이는 머신 러닝 모델이 훈련 데이터보다 새로운 데이터에서 성능이 낮아지는 경향을 말합니다. 과대적합은 3장에서 자세하게 논의하겠습니다.

이것으로 첫 번째 예제가 마무리되었습니다. 20줄 미만의 파이썬 코드로 손글씨 숫자를 분류하는 신경망을 만들고 훈련시켰습니다. 다음 장에서 여기서 보았던 코드 하나하나를 상세하게 설명하고 이들이 의미하는 바를 명확하게 설명하겠습니다. 이제 텐서, 신경망에 주입하는 데이터의 저장 형태, 층을 만들어주는 텐서 연산, 신경망을 훈련 샘플로부터 학습시키는 경사 하강법에 대해 알아보겠습니다.

## TODO 1: 은닉층의 유닛수
1. 첫 번째 은닉층의 유닛수를 64로 바꾸면 테스트 데이터 정확도는?
2. 첫 번째 은닉층의 유닛수를 1024로 바꾸면 테스트 데이터 정확도는?

In [15]:
# TODO: 64일 때 테스트 정확도 출력
from tensorflow.keras import optimizers
network = models.Sequential()
network.add(layers.Input(shape=(28 * 28,)))
network.add(layers.Dense(64, activation='relu')) #TODO
network.add(layers.Dense(10, activation='softmax'))

network.compile(optimizer='rmsprop',
                loss='categorical_crossentropy',
                metrics=['accuracy'])

network.fit(norm_train_images, cat_train_labels, epochs=5, batch_size=128)
test_loss, test_acc = network.evaluate(norm_test_images, cat_test_labels)
print('test_acc:', test_acc)

Epoch 1/5
469/469 ━━━━━━━━━━━━━━━━━━━━ 2s 2ms/step - accuracy: 0.8296 - loss: 0.6434
Epoch 2/5
469/469 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - accuracy: 0.9408 - loss: 0.2069
Epoch 3/5
469/469 ━━━━━━━━━━━━━━━━━━━━ 2s 3ms/step - accuracy: 0.9550 - loss: 0.1543
Epoch 4/5
469/469 ━━━━━━━━━━━━━━━━━━━━ 3s 4ms/step - accuracy: 0.9645 - loss: 0.1211
Epoch 5/5
469/469 ━━━━━━━━━━━━━━━━━━━━ 2s 2ms/step - accuracy: 0.9700 - loss: 0.1054
313/313 ━━━━━━━━━━━━━━━━━━━━ 1s 1ms/step - accuracy: 0.9612 - loss: 0.1360
test_acc: 0.965399980545044


In [16]:
# TODO: 1024일 때 테스트 정확도 출력
network.add(layers.Dense(1024, activation='relu'))
network.fit(norm_train_images, cat_train_labels, epochs=5, batch_size=128)
test_loss, test_acc = network.evaluate(norm_test_images, cat_test_labels)
print('test_acc:', test_acc)

Epoch 1/5
469/469 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - accuracy: 0.9742 - loss: 0.0902
Epoch 2/5
469/469 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - accuracy: 0.9764 - loss: 0.0800
Epoch 3/5
469/469 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - accuracy: 0.9798 - loss: 0.0714
Epoch 4/5
469/469 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - accuracy: 0.9824 - loss: 0.0631
Epoch 5/5
469/469 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - accuracy: 0.9842 - loss: 0.0560
313/313 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - accuracy: 0.9714 - loss: 0.1002
test_acc: 0.9750999808311462


## TODO 2: 은닉층의 활성화 함수
1. 첫 번째 은닉층의 활성화 함수를 sigmoid로 바꾸면 테스트 데이터 정확도는?
2. 첫 번째 은닉층의 활성화 함수를 tanh로 바꾸면 테스트 데이터 정확도는?

In [18]:
# TODO: sigmoid일 때 테스트 정확도 출력
from tensorflow.keras import optimizers
network = models.Sequential()
network.add(layers.Input(shape=(28 * 28,)))
network.add(layers.Dense(512, activation='sigmoid'))#TODO
network.add(layers.Dense(10, activation='softmax'))

network.compile(optimizer='rmsprop',
                loss='categorical_crossentropy',
                metrics=['accuracy'])

network.fit(norm_train_images, cat_train_labels, epochs=5, batch_size=128)
test_loss, test_acc = network.evaluate(norm_test_images, cat_test_labels)
print('test_acc:', test_acc)

Epoch 1/5
469/469 ━━━━━━━━━━━━━━━━━━━━ 5s 8ms/step - accuracy: 0.8096 - loss: 0.7378
Epoch 2/5
469/469 ━━━━━━━━━━━━━━━━━━━━ 6s 10ms/step - accuracy: 0.9189 - loss: 0.2797
Epoch 3/5
469/469 ━━━━━━━━━━━━━━━━━━━━ 4s 8ms/step - accuracy: 0.9322 - loss: 0.2309
Epoch 4/5
469/469 ━━━━━━━━━━━━━━━━━━━━ 5s 10ms/step - accuracy: 0.9465 - loss: 0.1864
Epoch 5/5
469/469 ━━━━━━━━━━━━━━━━━━━━ 5s 10ms/step - accuracy: 0.9555 - loss: 0.1515
313/313 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - accuracy: 0.9536 - loss: 0.1585
test_acc: 0.9602000117301941


In [22]:
# TODO: tanh일 때 테스트 정확도 출력
network.add(layers.Dense(512, activation='tanh'))
network.fit(norm_train_images, cat_train_labels, epochs=5, batch_size=128)
test_loss, test_acc = network.evaluate(norm_test_images, cat_test_labels)
print('test_acc:', test_acc)

Epoch 1/5
469/469 ━━━━━━━━━━━━━━━━━━━━ 4s 8ms/step - accuracy: 0.9813 - loss: 0.0642
Epoch 2/5
469/469 ━━━━━━━━━━━━━━━━━━━━ 5s 9ms/step - accuracy: 0.9842 - loss: 0.0563
Epoch 3/5
469/469 ━━━━━━━━━━━━━━━━━━━━ 5s 9ms/step - accuracy: 0.9857 - loss: 0.0494
Epoch 4/5
469/469 ━━━━━━━━━━━━━━━━━━━━ 5s 8ms/step - accuracy: 0.9874 - loss: 0.0458
Epoch 5/5
469/469 ━━━━━━━━━━━━━━━━━━━━ 7s 11ms/step - accuracy: 0.9890 - loss: 0.0402
313/313 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - accuracy: 0.9741 - loss: 0.0775
test_acc: 0.9778000116348267


## TODO 3: Softmax

아래 코드블록을 사용하여 답변하세요.
1. [-1, 0, 1]에서 softmax 결과 0이 선택될 확률은?
2. [1, 2, 3]에서 softmax 결과 3이 선택될 확률은?
3. [10, 20, 30]에서 softmax 결과 30이 선택될 확률은?

In [25]:
# TODO: 위의 질문에 대한 답을 출력하세요.
# Softmax 함수는 모든 범위의 실수 값들을 받아들여 확률로 변환 가능한 장점이 있습니다.

from scipy.special import softmax

# 1번
print(softmax([-1,0,1])[1])

# 2번
print(softmax([1,2,3])[2])

# 3번
print(softmax([10, 20, 30])[2])

0.24472847105479764
0.6652409557748218
0.999954600070331


## TODO 4: Optimizer
https://keras.io/api/optimizers/sgd/
1. optimizer에 SGD()를 썼을 때 테스트 정확도는?
2. optimizer에 SGD(learning_rate=0.1, momentum=0.9, nesterov=True)를 썼을 때 테스트 정확도는?
3. optimizer에 Adam()을 썼을 때 테스트 정확도는?

In [28]:
# TODO: SGD()를 사용했을 때의 테스트 정확도 출력
from tensorflow.keras import optimizers
network = models.Sequential()
network.add(layers.Input(shape=(28 * 28,)))
network.add(layers.Dense(512, activation='relu'))
network.add(layers.Dense(10, activation='softmax'))

network.compile(optimizer=optimizers.SGD(),
                loss='categorical_crossentropy',
                metrics=['accuracy']) #TODO

network.fit(norm_train_images, cat_train_labels, epochs=5, batch_size=128)
test_loss, test_acc = network.evaluate(norm_test_images, cat_test_labels)
print('test_acc:', test_acc)

Epoch 1/5
469/469 ━━━━━━━━━━━━━━━━━━━━ 4s 7ms/step - accuracy: 0.6103 - loss: 1.5432
Epoch 2/5
469/469 ━━━━━━━━━━━━━━━━━━━━ 5s 6ms/step - accuracy: 0.8664 - loss: 0.5680
Epoch 3/5
469/469 ━━━━━━━━━━━━━━━━━━━━ 3s 6ms/step - accuracy: 0.8873 - loss: 0.4359
Epoch 4/5
469/469 ━━━━━━━━━━━━━━━━━━━━ 6s 9ms/step - accuracy: 0.8991 - loss: 0.3790
Epoch 5/5
469/469 ━━━━━━━━━━━━━━━━━━━━ 4s 6ms/step - accuracy: 0.9044 - loss: 0.3480
313/313 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - accuracy: 0.8983 - loss: 0.3633
test_acc: 0.9129999876022339


In [30]:
# TODO: SGD(learning_rate=0.1, momentum=0.9, nesterov=True)를 사용했을 때의 테스트 정확도 출력
network.compile(optimizer=optimizers.SGD(learning_rate=0.1, momentum=0.9, nesterov=True),
                loss='categorical_crossentropy',
                metrics=['accuracy'])

network.fit(norm_train_images, cat_train_labels, epochs=5, batch_size=128)
test_loss, test_acc = network.evaluate(norm_test_images, cat_test_labels)
print('test_acc:', test_acc)

Epoch 1/5
469/469 ━━━━━━━━━━━━━━━━━━━━ 4s 8ms/step - accuracy: 0.9261 - loss: 0.2459
Epoch 2/5
469/469 ━━━━━━━━━━━━━━━━━━━━ 6s 10ms/step - accuracy: 0.9741 - loss: 0.0888
Epoch 3/5
469/469 ━━━━━━━━━━━━━━━━━━━━ 4s 8ms/step - accuracy: 0.9829 - loss: 0.0560
Epoch 4/5
469/469 ━━━━━━━━━━━━━━━━━━━━ 5s 7ms/step - accuracy: 0.9882 - loss: 0.0377
Epoch 5/5
469/469 ━━━━━━━━━━━━━━━━━━━━ 5s 10ms/step - accuracy: 0.9933 - loss: 0.0248
313/313 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - accuracy: 0.9775 - loss: 0.0781
test_acc: 0.9822999835014343


In [32]:
# TODO: Adam()을 사용했을 때의 테스트 정확도 출력
network.compile(optimizer=optimizers.Adam(),
                loss='categorical_crossentropy',
                metrics=['accuracy'])

network.fit(norm_train_images, cat_train_labels, epochs=5, batch_size=128)
test_loss, test_acc = network.evaluate(norm_test_images, cat_test_labels)
print('test_acc:', test_acc)

Epoch 1/5
469/469 ━━━━━━━━━━━━━━━━━━━━ 6s 12ms/step - accuracy: 0.9923 - loss: 0.0253
Epoch 2/5
469/469 ━━━━━━━━━━━━━━━━━━━━ 4s 9ms/step - accuracy: 0.9967 - loss: 0.0135
Epoch 3/5
469/469 ━━━━━━━━━━━━━━━━━━━━ 5s 9ms/step - accuracy: 0.9982 - loss: 0.0082
Epoch 4/5
469/469 ━━━━━━━━━━━━━━━━━━━━ 6s 11ms/step - accuracy: 0.9974 - loss: 0.0089
Epoch 5/5
469/469 ━━━━━━━━━━━━━━━━━━━━ 4s 9ms/step - accuracy: 0.9979 - loss: 0.0074
313/313 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - accuracy: 0.9762 - loss: 0.0876
test_acc: 0.9800000190734863


## TODO 5: 정규화의 영향

1. 이미지들을 정규화하지 않고 학습시켰을 때 테스트 정확도는?

In [35]:
# TODO: [0, 1]로의 정규화 없이 이미지를 일차원 배열로 reshape만 수행했을 때의 테스트 정확도 출력
conv_train_images = train_images.reshape((60000, 28 * 28))
conv_test_images = test_images.reshape((10000,28*28))

from tensorflow.keras import optimizers
network = models.Sequential()
network.add(layers.Input(shape=(28 * 28,)))
network.add(layers.Dense(512, activation='relu'))
network.add(layers.Dense(10, activation='softmax'))

network.compile(optimizer='rmsprop',
                loss='categorical_crossentropy',
                metrics=['accuracy'])

network.fit(conv_train_images,cat_train_labels , epochs=5, batch_size=128)
test_loss, test_acc = network.evaluate(conv_test_images,cat_test_labels )
print('test_acc:', test_acc)

Epoch 1/5
469/469 ━━━━━━━━━━━━━━━━━━━━ 5s 9ms/step - accuracy: 0.8357 - loss: 15.4584
Epoch 2/5
469/469 ━━━━━━━━━━━━━━━━━━━━ 5s 10ms/step - accuracy: 0.9510 - loss: 0.7357
Epoch 3/5
469/469 ━━━━━━━━━━━━━━━━━━━━ 4s 8ms/step - accuracy: 0.9661 - loss: 0.4290
Epoch 4/5
469/469 ━━━━━━━━━━━━━━━━━━━━ 4s 9ms/step - accuracy: 0.9708 - loss: 0.3784
Epoch 5/5
469/469 ━━━━━━━━━━━━━━━━━━━━ 5s 9ms/step - accuracy: 0.9765 - loss: 0.2864
313/313 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - accuracy: 0.9573 - loss: 0.8653
test_acc: 0.9617000222206116


## TODO 6: 최적화

1. 자유롭게 MLP (Multi-layer Perceptron)를 만들어서 Test 정확도를 98.2% 이상으로 만들어보세요. (층 추가 가능)

In [40]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.datasets import mnist
from tensorflow.keras.utils import to_categorical
(train_images, train_labels), (test_images, test_labels) = mnist.load_data()
train_images = train_images.reshape((60000, 28 * 28)).astype('float32') / 255.0
test_images = test_images.reshape((10000, 28 * 28)).astype('float32') / 255.0
train_labels = to_categorical(train_labels, 10)
test_labels = to_categorical(test_labels, 10)
model = Sequential([
    Dense(512, activation='relu', input_shape=(28 * 28,)),
    Dropout(0.2),
    Dense(256, activation='relu'),
    Dropout(0.2),
    Dense(128, activation='relu'),
    Dense(10, activation='softmax')
])
model.compile(optimizer=Adam(learning_rate=0.001),
              loss='categorical_crossentropy',
              metrics=['accuracy'])
model.fit(train_images, train_labels, epochs=25, batch_size=128, verbose=2, validation_split=0.2)
test_loss, test_accuracy = model.evaluate(test_images, test_labels, verbose=0)
print(f"Test Accuracy: {test_accuracy * 100:.2f}%")

Epoch 1/25
375/375 - 6s - 17ms/step - accuracy: 0.9076 - loss: 0.3053 - val_accuracy: 0.9615 - val_loss: 0.1255
Epoch 2/25
375/375 - 6s - 17ms/step - accuracy: 0.9640 - loss: 0.1210 - val_accuracy: 0.9677 - val_loss: 0.1019
Epoch 3/25
375/375 - 5s - 13ms/step - accuracy: 0.9732 - loss: 0.0866 - val_accuracy: 0.9758 - val_loss: 0.0823
Epoch 4/25
375/375 - 6s - 16ms/step - accuracy: 0.9783 - loss: 0.0684 - val_accuracy: 0.9772 - val_loss: 0.0769
Epoch 5/25
375/375 - 10s - 27ms/step - accuracy: 0.9821 - loss: 0.0558 - val_accuracy: 0.9775 - val_loss: 0.0815
Epoch 6/25
375/375 - 10s - 25ms/step - accuracy: 0.9846 - loss: 0.0472 - val_accuracy: 0.9788 - val_loss: 0.0803
Epoch 7/25
375/375 - 6s - 17ms/step - accuracy: 0.9863 - loss: 0.0418 - val_accuracy: 0.9756 - val_loss: 0.0871
Epoch 8/25
375/375 - 9s - 24ms/step - accuracy: 0.9882 - loss: 0.0359 - val_accuracy: 0.9779 - val_loss: 0.0833
Epoch 9/25
375/375 - 6s - 17ms/step - accuracy: 0.9884 - loss: 0.0359 - val_accuracy: 0.9761 - val_los

##TODO 7: 마무리

오늘 실습에서 인공신경망의 성능에 영향을 준 요인들을 나열해보세요.

In [41]:
# A: 데이터 양, 데이터 정규화, 데이터 최적화, 모델 활성화 함수, Softmax, Optimizer, SGD, Adam RMSprop, EPpochs, 은닉층과 유닛의 개수